In [2]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP
import src.Fluid_CP as FCP
import src.Compressormodel_SP as CSP

**Assumptions**

- water basin and pasta can be modelled as 0D --> no temperature gradients inside water and pasta

- temperatures and pressure dependency of the properties of water and the pasta can be neglected

- subcritical heat pump process

- The pasta water is the heat sink

- Make sure the heat pump meets usual technical constraints (e.g., superheating) and is 
in COP-optimal operation at each point in time. 


In [ ]:
# parameters

## basin geometry



## cooking pasta

Tw_morning = 20 #degree Celsius

Tp_before = 7 #degree celsius

P_batch = 2 # kg --> 0.5 kg per basket

tc = 7 #minutes --> pasta is cooked for 7 minutes per batch

Tp_after = 85 #degree celsius

tb = 5 #minutes --> time between cooking batches

Tw_op_min = 85 #degree celsius --> minimal temperature during cooking

Tw_op_max = 93 #degree celsius --> maximal temperature during cooking

## other

R_env_w = 30 #K/kW --> evaporation of the cooking  water is neglected

R_p_w = 15 #K/kW 

c_pw = 4.18 #kj/kgK

c_pp = 3.5 #kj/kgK

## heatpump process

Tamb = 20 #degree celsius --> air temperature in cross flow

## compressor

D = [35,40,45,50] #possible compressor sizes in mm

## Heat exchanger

d_outer = 6.35 #mm

d_inner = 5.55 #mm

## Heat transfer coefficients

h_r_li = 1000 #W/(m^2*K)

h_r_2ph = 2000 #W/(m^2*K)

h_r_va = 200 #W/(m^2*K)

h_w = 1000 #W/(m^2*K)


In [4]:
# empirical calculation of resulting heatpump COP

def COP_res(COP_inner, Q_demand, Q_heatpump):
    return COP_inner * ((Q_demand / Q_heatpump) / 0.9*(Q_demand / Q_heatpump) + 0.1)

